In [1]:
import numpy as np

In [2]:
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    
    return distribution / np.sum(distribution)

In [3]:
import keras
import numpy as np

Using TensorFlow backend.


In [4]:
path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

text = open(path).read().lower()

print('Corpus length: {}'.format(len(text)))

606208/600901 [==============================] - 1s 1us/step
Corpus length: 600893


In [53]:
text

'preface\n\n\nsupposing that truth is a woman--what then? is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to truth, have been unskilled and unseemly methods for\nwinning a woman? certainly she has never allowed herself to be won; and\nat present every kind of dogma stands with sad and discouraged mien--if,\nindeed, it stands at all! for there are scoffers who maintain that it\nhas fallen, that all dogma lies on the ground--nay more, that it is at\nits last gasp. but to speak seriously, there are good grounds for hoping\nthat all dogmatizing in philosophy, whatever solemn, whatever conclusive\nand decided airs it has assumed, may have been only a noble puerilism\nand tyronism; and probably the time is at hand when it will be once\nand again understood what has actually sufficed for the basis of such\

In [7]:
maxlen = 60 # extraxt sequences of n characters
step = 3 # sample new seq every n characters

In [8]:
sentences = []
next_chars = []

In [24]:
for i in range(0, len(text) - maxlen, step): # iterates by step size
    sentences.append(text[i: i + maxlen]) # get maxlen amount of charachters
    next_chars.append(text[i + maxlen])
    
print('Number of sequences:', len(sentences))

Number of sequences: 400556


In [25]:
chars = sorted(list(set(text))) # list of unique characters
print('Unique characters:', len(chars))

char_indices = dict((char, chars.index(char)) for char in chars) # maps char with index

Unique characters: 57


In [39]:
print('Vectorization...')

x = np.zeros((len(sentences), maxlen, len(chars))) # (sentences)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [45]:
from keras import layers

In [46]:
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [47]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [48]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [52]:
import random
import sys
for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen] 
    print('--- Generating with seed: "' + generated_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)

epoch 1
Epoch 1/1
400556/400556 [==============================] - 232s 580us/step - loss: 1.8248
--- Generating with seed: "the most attractive, ensnaring,
and exquisite element in tho"
------ temperature: 0.2
the most attractive, ensnaring,
and exquisite element in those the possible the contend the case of the still the still and a sufferss of the presention of the strings the contemption of the suffers and the good and prote itself in the call the art the case the presention of the call the still of the proble the still the called the presention of the call the presention of the still and the string and protess of the still the called the presention of the st------ temperature: 0.5
and protess of the still the called the presention of the still the goded and strongers of the case the religion the call the whole made an olders which he solef the self-all the wors and souling the decisely the sute and action is an ablither the spirit of his desplition of the greated so belief really 

/Users/hao/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


uth other allook
fear omnegled need of alls?


1hinthang, couch to dw astrains showth
cooply las paragers? sn
afford condition, and owpor all whomilere,
however, alloquelsy of whome does exist llibepher
arounting norrable through has
as "good.=--moraly theerorepoch 4
Epoch 1/1
400556/400556 [==============================] - 241s 601us/step - loss: 1.4470
--- Generating with seed: "ch is adapted to a
tender, many-sided, and very fastidious c"
------ temperature: 0.2
ch is adapted to a
tender, many-sided, and very fastidious conscience to the sense of the superinger and the strong and of the supposes and and should the superinger the an instinctic just the superinger and of the strong and strong the conscience and in the strict the strong and the superinger and the sense and the conscience the degree and a consideres to the conscience and of the supposing the conscienne and also the sense the strong and in the conscien------ temperature: 0.5
conscienne and also the sense the strong and 

the his into, utile of the enor, and ru theinal ins an reall pasity,--hitherd in thesgise,bhere, orse a------ temperature: 1.2
inal ins an reall pasity,--hitherd in thesgise,bhere, orse arour , which of for musuaw tass in a smeed silk heravidnificedgly your ander to rematises wear it far longian he lavily,
whole as they that (creall
mistos, the, sifd soult blitato rests, exer he protaussly whech ? he firse thegient
frevaritalll all admuland pbase into away bethar hard lishes, "ak;
he subbirentie monesscouial arhul solitudia-commjces thousayly bodisupty, many,er, howevoead quider iepoch 8
Epoch 1/1
400556/400556 [==============================] - 238s 594us/step - loss: 1.6350
--- Generating with seed: "cession of philosophical systems,
just as the way seems barr"
------ temperature: 0.2
cession of philosophical systems,
just as the way seems barrxing the have a may be moralist and the will individ to the individy of the morality of the word to all the have in the has senses to the comp

 the new and himself and places of the frant of his amouth other known adngeninements, was nature of prediced path in
enough to not path the discence in orders which is alone when all instincts, the geniuule, belief is no rensight and when from which
it, perhawer, above it and angody--should beouthy all that one to philosopher worldly softer reality,
graniful of bad for
allitthtness of history princible before and paly religate
clate most messetion, it tas------ temperature: 1.2
ncible before and paly religate
clate most messetion, it taste,
as laping to
goodnow by "from
licks agelys clignce
of
inhurtimer nard
faussied and vilipsm. the extentest
to german world, but as lad fordner whenks sawndo
known on--the
persons like
atseductian isumucage
gratekly, the
filu, the old fromd: "one pathtllyful
atmadited madating portions
muct not him to
lookage
: in it, a gerble, whys flye accapacitude, anrower; can ritionss.." these woinhful; manepoch 12
Epoch 1/1
400556/400556 [======================

KeyboardInterrupt: 